# USGS Metadata Scrape

In [152]:
import pandas as pd
import numpy as np
import requests
import lxml.html as lh
from bs4 import BeautifulSoup


## This code pulls stream, spring, and groundwater well gauge site names, site #s, locations, types of parameters, lengths of record, # of samples, etc from USGS Site Web Service within our chosen HUCs:


    Accounting Unit 031102 -- Suwannee: The Suwannee River Basin.
                              Florida, Georgia.
                                Area =     9930 sq.mi.

      Cataloging Units  03110201 -- Upper Suwannee.  Florida, Georgia.
                                      Area =     2720 sq.mi.
                        03110202 -- Alapaha. Florida, Georgia.
                                      Area =     1840 sq.mi.
                        03110203 -- withlacoochee. Florida, Georgia.
                                      Area =     1510 sq.mi.
                        03110204 -- Little. Georgia.
                                      Area =      884 sq.mi.
                        03110205 -- Lower Suwannee. Florida.
                                      Area =     1590 sq.mi.
                        03110206 -- Santa Fe. Florida.
                                      Area =     1390 sq.mi.
                                      


### Stream Gauge Sites

#### The following cell pulls the metadata for all stream gauges within the Suwannee River Basin and creates a pandas dataframe. The first 5 rows of the dataframe are displayed below.

In [153]:
# URL generated by the USGS Site Web Service. Count number is for # of daily values.
stream_meta_full = pd.read_csv("https://waterservices.usgs.gov/nwis/site/?format=rdb&huc=03110201,03110202,03110203,03110204,03110205,03110206&startDT=1980-01-01&endDT=2021-06-08&outputDataTypeCd=dv&siteType=ST&siteStatus=all", sep='\t',skiprows=(42),header=(0))

stream_meta_df = stream_meta_full
stream_meta_df.drop([0],axis=0,inplace=True)
stream_meta_df.head()


,agency_cd,site_no,station_nm,site_tp_cd,dec_lat_va,dec_long_va,coord_acy_cd,dec_coord_datum_cd,alt_va,alt_acy_va,...,stat_cd,ts_id,loc_web_ds,medium_grp_cd,parm_grp_cd,srs_id,access_cd,begin_date,end_date,count_nu
1,USGS,02314274,"SUWANNEE RIVER (AT SILL) NEAR FARGO, GA",ST,30.80410984,-82.4173514,S,NAD83,117,.5,...,00001,26363,NaN,wat,NaN,1645423,0,1999-09-30,2003-09-29,1356
2,USGS,02314274,"SUWANNEE RIVER (AT SILL) NEAR FARGO, GA",ST,30.80410984,-82.4173514,S,NAD83,117,.5,...,00002,26364,NaN,wat,NaN,1645423,0,1999-09-30,2003-09-29,1356
3,USGS,02314274,"SUWANNEE RIVER (AT SILL) NEAR FARGO, GA",ST,30.80410984,-82.4173514,S,NAD83,117,.5,...,00003,26365,NaN,wat,NaN,1645423,0,1999-09-30,2003-09-29,1384
4,USGS,023142741,NORTH FORK SUWANNEE RIVER AT SILL NEAR FARGO GA,ST,30.81633217,-82.41346259,S,NAD83,117.00,.5,...,00006,26369,NaN,wat,NaN,1644459,0,2000-09-30,2001-09-18,354
5,USGS,023142741,NORTH FORK SUWANNEE RIVER AT SILL NEAR FARGO GA,ST,30.81633217,-82.41346259,S,NAD83,117.00,.5,...,00001,26366,NaN,wat,NaN,1645423,0,1998-12-02,2003-09-29,1694


#### The following cell determines which sites have more than 10 daily values recorded, prints the site numbers, and counts how many sites there are that fit that description.

In [154]:
stream_meta_df['count_nu'] = stream_meta_df['count_nu'].astype(int)
stream_meta_df = stream_meta_df.loc[stream_meta_df['count_nu'] > 10]
num_sites = stream_meta_df.site_no.unique()
print(num_sites)
len(num_sites)

['02314274' '023142741' '02314495' '02314500' '02314986' '02315000'
 '02315005' '02315200' '02315392' '02315500' '02315520' '02315550'
 '02315626' '02315920' '02316000' '02317500' '02317620' '02317748'
 '023177483' '02317755' '023177554' '02317797' '02318000' '02318380'
 '02318500' '02318700' '02319000' '02319300' '02319394' '02319500'
 '02319800' '02320000' '02320500' '02320700' '02321000' '02321500'
 '02321898' '02321900' '02321958' '02321975' '02322016' '02322049'
 '02322050' '02322500' '02322616' '02322700' '02322703' '02322800'
 '02323000' '02323500' '02323567' '02323590' '02323592' '291652083064100'
 '291841083070800' '291842083085100' '291930083082800' '302307083101400'
 '302556082433800' '302623082434200']


60

#### A list of unique parameter codes measured by the stream gauges is generated. Counts of each parameter are shown.

In [155]:
num_parms = stream_meta_df.parm_cd.unique()
print(num_parms)
stream_meta_df['parm_cd'].value_counts()

['00060' '00045' '00065' '00010' '00095' '00300' '00400' '99133' '72192'
 '80154' '80155' '63680' '63160' '72020' '00480' '63158' '72137']


00010    95
00065    72
00095    65
00060    55
00045    26
00480    22
72020    10
99133     9
63160     8
00400     6
00300     6
63680     3
72192     2
72137     1
80154     1
80155     1
63158     1
Name: parm_cd, dtype: int64

#### Parameter codes from the USGS are pulled down from the web and saved as a dataframe. Column headings are changed. The first 5 rows of the table are shown.

In [156]:
usgs_parm_codes = 'https://nwis.waterdata.usgs.gov/usa/nwis/pmcodes?radio_pm_search=param_group&pm_group=All+--+include+all+parameter+groups&pm_search=&casrn_search=&srsname_search=&format=html_table&show=parameter_group_nm&show=parameter_nm&show=casrn&show=srsname&show=parameter_units'
usgs_parm_codes_req = requests.get(usgs_parm_codes)
usgs_parm_codes_table = pd.read_html(usgs_parm_codes_req.text, match='1', header=0,converters={'Parameter Code': lambda x: str(x)})
usgs_parm_codes_df = usgs_parm_codes_table[0]
usgs_parm_codes_df.rename(columns={'Parameter Code': 'parm_cd', 
                            'Group Name': 'grp_nm', 'Parameter Name/Description':  'parm_desc', 
                            'Parameter Unit': 'parm_unit'}, inplace = True)
usgs_parm_codes_df.head()


,parm_cd,grp_nm,parm_desc,CASRN,SRSName,parm_unit
0,00001,Information,"Location in cross section, distance from right...",NaN,NaN,ft
1,00002,Information,"Location in cross section, distance from right...",NaN,NaN,%
2,00003,Information,"Sampling depth, feet",NaN,NaN,ft
3,00005,Information,"Location in cross section, fraction of total d...",NaN,NaN,%
4,00008,Information,Sample accounting number,NaN,NaN,nu


In [157]:
# Use line below to find the name and description of a given parameter code.
#usgs_parm_codes_df.loc[usgs_parm_codes_df['parm_cd']=='00631']

#### Stream site parameter codes are compared the the USGS master list and a table of the parameters measured at the sites is shown. 

In [158]:
#avail_parms = usgs_parm_codes_df.loc[usgs_parm_codes_df['parm_cd'] == num_parms]
avail_parm_codes_stream = np.where(usgs_parm_codes_df['parm_cd'].isin(num_parms))
avail_parms_stream = usgs_parm_codes_df.iloc[avail_parm_codes_stream]
display(avail_parms_stream)
stream_meta_df['parm_cd'].value_counts()

,parm_cd,grp_nm,parm_desc,CASRN,SRSName,parm_unit
1646,00010,Physical,"Temperature, water, degrees Celsius",NaN,"Temperature, water",deg C
1660,00045,Physical,"Precipitation, total, inches",NaN,Precipitation,in
1673,00060,Physical,"Discharge, cubic feet per second",NaN,"Stream flow, mean. daily",ft3/s
1677,00065,Physical,"Gage height, feet",NaN,"Height, gage",ft
1692,00095,Physical,"Specific conductance, water, unfiltered, micro...",NaN,Specific conductance,uS/cm @25C
1733,00400,Physical,"pH, water, unfiltered, field, standard units",NaN,pH,std units
1739,00480,Physical,"Salinity, water, unfiltered, parts per thousand",NaN,Salinity,ppth
1999,63158,Physical,"Stream water level elevation above NGVD 1929, ...",NaN,NaN,ft
2001,63160,Physical,"Stream water level elevation above NAVD 1988, ...",NaN,NaN,ft
2008,63680,Physical,"Turbidity, water, unfiltered, monochrome near ...",NaN,Turbidity,FNU


00010    95
00065    72
00095    65
00060    55
00045    26
00480    22
72020    10
99133     9
63160     8
00400     6
00300     6
63680     3
72192     2
72137     1
80154     1
80155     1
63158     1
Name: parm_cd, dtype: int64

#### Changing the number within the ' ' below will give a list of the stream sites that measure that parameter. Available codes to choose from are shown in table above.

In [159]:
display(stream_meta_df.loc[stream_meta_df['parm_cd'] == '99133'])

,agency_cd,site_no,station_nm,site_tp_cd,dec_lat_va,dec_long_va,coord_acy_cd,dec_coord_datum_cd,alt_va,alt_acy_va,...,stat_cd,ts_id,loc_web_ds,medium_grp_cd,parm_grp_cd,srs_id,access_cd,begin_date,end_date,count_nu
68,USGS,02315626,"ALAPAHA RISE ABOVE SW 68TH DRIVE NEAR JASPER, FL",ST,30.43944444,-83.0892778,5,NAD83,00.00,001,...,00001,26414,NaN,wat,NaN,701177,0,2015-06-18,2016-09-30,463
69,USGS,02315626,"ALAPAHA RISE ABOVE SW 68TH DRIVE NEAR JASPER, FL",ST,30.43944444,-83.0892778,5,NAD83,00.00,001,...,00002,26415,NaN,wat,NaN,701177,0,2015-06-18,2016-09-30,463
70,USGS,02315626,"ALAPAHA RISE ABOVE SW 68TH DRIVE NEAR JASPER, FL",ST,30.43944444,-83.0892778,5,NAD83,00.00,001,...,00003,26416,NaN,wat,NaN,701177,0,2015-06-18,2016-09-30,463
271,USGS,02322700,"ICHETUCKNEE R @ HWY27 NR HILDRETH, FL",ST,29.95273288,-82.7859573,F,NAD83,00.00,005,...,00001,165829,NaN,wat,NaN,701177,0,2016-11-16,2021-06-13,1412
272,USGS,02322700,"ICHETUCKNEE R @ HWY27 NR HILDRETH, FL",ST,29.95273288,-82.7859573,F,NAD83,00.00,005,...,00002,165830,NaN,wat,NaN,701177,0,2016-11-16,2021-06-13,1412
273,USGS,02322700,"ICHETUCKNEE R @ HWY27 NR HILDRETH, FL",ST,29.95273288,-82.7859573,F,NAD83,00.00,005,...,00003,165831,NaN,wat,NaN,701177,0,2016-11-16,2021-06-13,1413
285,USGS,02322800,SANTA FE RIVER NR HILDRETH FLA,ST,29.91162076,-82.8604033,S,NAD83,3.50,.1,...,00001,26625,NaN,wat,NaN,701177,0,2013-07-10,2016-09-30,981
286,USGS,02322800,SANTA FE RIVER NR HILDRETH FLA,ST,29.91162076,-82.8604033,S,NAD83,3.50,.1,...,00002,26626,NaN,wat,NaN,701177,0,2013-07-10,2016-09-30,981
287,USGS,02322800,SANTA FE RIVER NR HILDRETH FLA,ST,29.91162076,-82.8604033,S,NAD83,3.50,.1,...,00003,26627,NaN,wat,NaN,701177,0,2012-09-27,2016-09-30,1169


### Spring Gauge Sites


#### The following cell pulls the metadata for all spring gauges within the Suwannee River Basin and creates a pandas dataframe. The first 5 rows of the dataframe are displayed below.

In [160]:
spring_meta_full = pd.read_csv("https://waterservices.usgs.gov/nwis/site/?format=rdb&huc=03110201,03110202,03110203,03110204,03110205,03110206&startDT=1980-01-01&endDT=2021-06-09&outputDataTypeCd=dv&siteType=SP&siteStatus=all", sep='\t',skiprows=(42),header=(0))

spring_meta_df = spring_meta_full
spring_meta_df.drop([0],axis=0,inplace=True)
spring_meta_df.head()


,agency_cd,site_no,station_nm,site_tp_cd,dec_lat_va,dec_long_va,coord_acy_cd,dec_coord_datum_cd,alt_va,alt_acy_va,...,stat_cd,ts_id,loc_web_ds,medium_grp_cd,parm_grp_cd,srs_id,access_cd,begin_date,end_date,count_nu
1,USGS,02315620,"HOLTON SPRINGS NEAR FT. UNION,FL",SP,30.43771795,-83.0573582,S,NAD83,NaN,NaN,...,00003,175008,NaN,wat,NaN,1645423,0,2015-06-18,2016-09-29,470
2,USGS,02315620,"HOLTON SPRINGS NEAR FT. UNION,FL",SP,30.43771795,-83.0573582,S,NAD83,NaN,NaN,...,00003,26399,NaN,wat,NaN,17164583,0,2015-06-18,2016-09-29,470
3,USGS,02319302,"MADISON BLUE SPRING NR BLUE SPRINGS, FL",SP,30.48049106,-83.2443097,F,NAD83,48,15,...,00001,222461,NaN,wat,NaN,1645597,0,2014-07-09,2021-06-13,1952
4,USGS,02319302,"MADISON BLUE SPRING NR BLUE SPRINGS, FL",SP,30.48049106,-83.2443097,F,NAD83,48,15,...,00002,222463,NaN,wat,NaN,1645597,0,2014-07-09,2021-06-13,1952
5,USGS,02319302,"MADISON BLUE SPRING NR BLUE SPRINGS, FL",SP,30.48049106,-83.2443097,F,NAD83,48,15,...,00003,172070,NaN,wat,NaN,1645597,0,2014-07-09,2021-06-13,1952


#### The following cell determines which sites have more than 10 daily values recorded, prints the site numbers, and counts how many sites there are that fit that description.

In [161]:
spring_meta_df['count_nu'] = spring_meta_df['count_nu'].astype(int)
spring_meta_df = spring_meta_df.loc[spring_meta_df['count_nu'] > 10]

num_sites = spring_meta_df.site_no.unique()
print(num_sites)
len(num_sites)

['02315620' '02319302' '02319520' '02319950' '02320250' '02322400'
 '02322685' '02322687' '02322688' '02322691' '02322694' '02322695'
 '02322698' '02323502' '02323566']


15

#### A list of unique parameter codes measured by the spring gauges is generated. Counts of each parameter are shown.

In [162]:
num_parms = spring_meta_df.parm_cd.unique()
print(num_parms)
print(len(num_parms))
spring_meta_df['parm_cd'].value_counts()

['00060' '00065' '00010' '00095' '00300' '00400' '63160' '99133']
8


00300    24
00095    24
00400    24
99133    24
00010    23
00065    15
00060    12
63160     4
Name: parm_cd, dtype: int64

#### Spring site parameter codes are compared the the USGS master list and a table of the parameters measured at the sites is shown.

In [163]:
avail_parm_codes_spring = np.where(usgs_parm_codes_df['parm_cd'].isin(num_parms))
avail_parms_spring = usgs_parm_codes_df.iloc[avail_parm_codes_spring]
display(avail_parms_spring)

,parm_cd,grp_nm,parm_desc,CASRN,SRSName,parm_unit
1646,00010,Physical,"Temperature, water, degrees Celsius",NaN,"Temperature, water",deg C
1673,00060,Physical,"Discharge, cubic feet per second",NaN,"Stream flow, mean. daily",ft3/s
1677,00065,Physical,"Gage height, feet",NaN,"Height, gage",ft
1692,00095,Physical,"Specific conductance, water, unfiltered, micro...",NaN,Specific conductance,uS/cm @25C
1733,00400,Physical,"pH, water, unfiltered, field, standard units",NaN,pH,std units
2001,63160,Physical,"Stream water level elevation above NAVD 1988, ...",NaN,NaN,ft
2740,00300,"Inorganics, Major, Non-metals","Dissolved oxygen, water, unfiltered, milligram...",7782-44-7,Oxygen,mg/l
3421,99133,Nutrient,"Nitrate plus nitrite, water, in situ, milligra...",NaN,Inorganic nitrogen (nitrate and nitrite),mg/l as N


#### Changing the number within the ' ' below will give a list of the spring sites that measure that parameter. Available codes to choose from are shown in table above.

In [164]:
display(spring_meta_df.loc[spring_meta_df['parm_cd'] == '00060'])

,agency_cd,site_no,station_nm,site_tp_cd,dec_lat_va,dec_long_va,coord_acy_cd,dec_coord_datum_cd,alt_va,alt_acy_va,...,stat_cd,ts_id,loc_web_ds,medium_grp_cd,parm_grp_cd,srs_id,access_cd,begin_date,end_date,count_nu
1,USGS,02315620,"HOLTON SPRINGS NEAR FT. UNION,FL",SP,30.43771795,-83.0573582,S,NAD83,NaN,NaN,...,00003,175008,NaN,wat,NaN,1645423,0,2015-06-18,2016-09-29,470
6,USGS,02319302,"MADISON BLUE SPRING NR BLUE SPRINGS, FL",SP,30.48049106,-83.2443097,F,NAD83,48,15,...,00003,174138,NaN,wat,NaN,1645423,0,2002-04-12,2021-06-13,6697
40,USGS,02319950,"BLUE SPRINGS NEAR DELL,FL",SP,30.126055,-83.2259712,S,NAD83,25.00,5,...,00003,224525,NaN,wat,NaN,1645423,0,2015-06-17,2021-06-13,2184
85,USGS,02322685,"ICHETUCKNEE HEAD SPRING NR HILDRETH, FL",SP,29.98388889,-82.7619444,S,NAD83,21.76,.01,...,00003,26587,NaN,wat,NaN,1645423,0,2002-02-07,2010-06-08,3044
87,USGS,02322687,"CEDAR HEAD SPRING NR HILDRETH, FL",SP,29.98305556,-82.75888889,S,NAD83,13.83,.01,...,00003,26589,NaN,wat,NaN,1645423,0,2002-02-11,2010-05-22,3023
92,USGS,02322688,"BLUE HOLE SPRING NR HILDRETH, FL",SP,29.9799552,-82.758457,S,NAD83,20.31,.01,...,00003,229570,NaN,wat,NaN,1645423,0,2002-02-05,2021-06-11,6893
107,USGS,02322691,"MISSION SPRINGS COMPLEX NR HILDRETH, FL",SP,29.97583333,-82.7583333,S,NAD83,0.51,.01,...,00003,26612,NaN,wat,NaN,1645423,0,2002-02-12,2010-05-06,3006
109,USGS,02322694,"DEVIL'S EYE SPRING NR HILDRETH, FL",SP,29.97333333,-82.7602778,S,NAD83,12.50,.01,...,00003,26614,NaN,wat,NaN,1645423,0,2002-02-12,2010-06-09,3040
111,USGS,02322695,"MILL POND SPRING NR HILDRETH, FL",SP,29.96638889,-82.76,S,NAD83,18.76,.01,...,00003,26616,NaN,wat,NaN,1645423,0,2002-02-14,2010-04-20,2988
113,USGS,02322698,"ICHETUCKNEE R A DAMPIER'S LANDING NR HILDRETH, FL",SP,29.96027778,-82.7722222,S,NAD83,8.62,.01,...,00003,26618,NaN,wat,NaN,1645423,0,2002-02-14,2012-09-29,3751


### Groundwater Well Sites


#### The following cell pulls the metadata for all groundwater wells within the Suwannee River Basin and creates a pandas dataframe. The first 5 rows of the dataframe are displayed below.

In [165]:
gw_meta_full = pd.read_csv("https://waterservices.usgs.gov/nwis/site/?format=rdb&huc=03110201,03110202,03110203,03110204,03110205,03110206&startDT=1980-01-01&endDT=2021-06-09&outputDataTypeCd=gw&siteType=GW,GW-CR,GW-EX,GW-HZ,GW-IW,GW-MW,GW-TH&siteStatus=all", sep='\t',skiprows=(42),header=(0))

gw_meta_df = gw_meta_full
gw_meta_df.drop([0],axis=0,inplace=True)
gw_meta_df.head()


,agency_cd,site_no,station_nm,site_tp_cd,dec_lat_va,dec_long_va,coord_acy_cd,dec_coord_datum_cd,alt_va,alt_acy_va,...,stat_cd,ts_id,loc_web_ds,medium_grp_cd,parm_grp_cd,srs_id,access_cd,begin_date,end_date,count_nu
1,USGS,292448083023801,GP-6 UFA Well,GW,29.41333333,-83.04388889,S,NAD83,9,4.3,...,NaN,0,NaN,wat,NaN,1642461,0,2009-05-22,2009-05-22,1
2,USGS,292448083023801,GP-6 UFA Well,GW,29.41333333,-83.04388889,S,NAD83,9,4.3,...,NaN,0,NaN,wat,NaN,1642461,0,2009-05-22,2009-05-22,1
3,USGS,292448083023801,GP-6 UFA Well,GW,29.41333333,-83.04388889,S,NAD83,9,4.3,...,NaN,0,NaN,wat,NaN,1642008,0,2009-05-22,2009-05-22,1
4,USGS,292507082560201,A J MIMMS(121420)SR 347 SW OF CHIEFLAND,GW,29.41885004,-82.9337331,S,NAD83,33.00,.01,...,NaN,0,NaN,wat,NaN,1642461,0,1986-09-17,2009-05-19,28
5,USGS,292507082560201,A J MIMMS(121420)SR 347 SW OF CHIEFLAND,GW,29.41885004,-82.9337331,S,NAD83,33.00,.01,...,NaN,0,NaN,wat,NaN,1642461,0,1986-09-17,2009-05-19,28


#### The following cell determines which sites have more than 5 groundwater levels recorded, prints the site numbers, and counts how many sites there are that fit that description.

In [166]:
gw_meta_df['count_nu'] = gw_meta_df['count_nu'].astype(int)
gw_meta_df = gw_meta_df.loc[gw_meta_df['count_nu'] > 10]

num_sites = gw_meta_df.site_no.unique()
print(num_sites)
len(num_sites)

['292507082560201' '292713082493601' '292831082445101' '292843082514201'
 '292921082583285' '292935083025402' '292957082593901' '294043082512501'
 '294306082234701' '294307082020903' '294313082024601' '294330082445001'
 '294530082232001' '294533082294401' '294538082592601' '294629082181301'
 '294656082594701' '294721082443001' '294743082543901' '294813082520501'
 '294839082230701' '294920082044501' '294928082355301' '295105082443301'
 '295114082393801' '295129082332601' '295130082243001' '295214082482501'
 '295354082513801' '295615082475401' '295618082440901' '295707082393701'
 '295737082480801' '295850082140301' '295907082423501' '300010082594201'
 '300019082355401' '300020082103001' '300101082245201' '300114083025901'
 '300143082565903' '300249083045301' '300319083065001' '300351083074801'
 '300400082585001' '300409082415001' '300412083070301' '300443083055701'
 '300455083051501' '300542082332101' '300612082363101' '300615082130501'
 '300629082030001' '300635082295901' '3007520824739

245

#### A list of unique parameter codes measured by the groundwater sites is generated. Counts of each parameter are shown.

In [167]:
num_parms = gw_meta_df.parm_cd.unique()
print(num_parms)
print(len(num_parms))
gw_meta_df['parm_cd'].value_counts()

['62610' '62611' '72019']
3


62610    245
72019    245
62611    245
Name: parm_cd, dtype: int64

#### Groundwater site parameter codes are compared the the USGS master list and a table of the parameters measured at the sites is shown.

In [168]:
avail_parm_codes_gw = np.where(usgs_parm_codes_df['parm_cd'].isin(num_parms))
avail_parms_gw = usgs_parm_codes_df.iloc[avail_parm_codes_gw]
display(avail_parms_gw)

,parm_cd,grp_nm,parm_desc,CASRN,SRSName,parm_unit
1950,62610,Physical,"Groundwater level above NGVD 1929, feet",NaN,NaN,ft
1951,62611,Physical,"Groundwater level above NAVD 1988, feet",NaN,NaN,ft
2059,72019,Physical,"Depth to water level, feet below land surface",NaN,Depth to water level below land surface,ft


#### Changing the number within the ' ' below will give a list of the groundwater sites that measure that parameter. Available codes to choose from are shown in table above.

In [169]:
display(gw_meta_df.loc[gw_meta_df['parm_cd'] == '62610'])

,agency_cd,site_no,station_nm,site_tp_cd,dec_lat_va,dec_long_va,coord_acy_cd,dec_coord_datum_cd,alt_va,alt_acy_va,...,stat_cd,ts_id,loc_web_ds,medium_grp_cd,parm_grp_cd,srs_id,access_cd,begin_date,end_date,count_nu
4,USGS,292507082560201,A J MIMMS(121420)SR 347 SW OF CHIEFLAND,GW,29.41885004,-82.9337331,S,NAD83,33.00,.01,...,NaN,0,NaN,wat,NaN,1642461,0,1986-09-17,2009-05-19,28
7,USGS,292713082493601,"H.E.MILLS NR CHIEFLAND,FL",GW,29.45385208,-82.8265073,S,NAD83,31,1.6,...,NaN,0,NaN,wat,NaN,1642461,0,1984-05-15,2009-05-19,39
10,USGS,292831082445101,DOT-27A,GW,29.47552064,-82.7473379,S,NAD83,49,1.6,...,NaN,0,NaN,wat,NaN,1642461,0,1998-04-29,1999-09-30,18
13,USGS,292843082514201,928251141 11S14E36 DRUMMOND LUMBER CO,GW,29.47885087,-82.86150889,S,NAD83,35.00,.1,...,NaN,0,NaN,wat,NaN,1642461,0,1961-02-08,1990-05-16,72
16,USGS,292921082583285,MANATEE SPRINGS STATE PARK,GW,29.4886611,-82.97523889,S,NAD83,16,.01,...,NaN,0,NaN,wat,NaN,1642461,0,2009-05-22,2021-06-09,60
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3253,USGS,315043083401901,16N006,GW,31.84545527,-83.6718355,F,NAD83,422,10,...,NaN,0,NaN,wat,NaN,1642461,0,1994-03-03,2013-04-24,20
3256,USGS,315059083350901,17N003,GW,31.8499003,-83.5857219,F,NAD83,430,10,...,NaN,0,NaN,wat,NaN,1642461,0,1994-03-10,2021-04-13,18
3259,USGS,315314083395501,16P004,GW,31.8873989,-83.6651687,F,NAD83,390.00,5,...,NaN,0,NaN,wat,NaN,1642461,0,1977-12-01,1994-11-01,14
3268,USGS,315657083273001,18P001,GW,31.95045496,-83.4576634,U,NAD83,367,10,...,NaN,0,NaN,wat,NaN,1642461,0,1966-02-16,1990-05-18,12


In [170]:
usgs_all_sites_df = pd.concat((stream_meta_df,spring_meta_df,gw_meta_df))

In [171]:
num_parms = usgs_all_sites_df.parm_cd.unique()
print(num_parms)
print(len(num_parms))
parm_instance_all = usgs_all_sites_df['parm_cd'].value_counts()

['00060' '00045' '00065' '00010' '00095' '00300' '00400' '99133' '72192'
 '80154' '80155' '63680' '63160' '72020' '00480' '63158' '72137' '62610'
 '62611' '72019']
20


In [172]:
avail_parm_codes_all = np.where(usgs_parm_codes_df['parm_cd'].isin(num_parms))
avail_parms_all = usgs_parm_codes_df.iloc[avail_parm_codes_all]

display(avail_parms_all)

,parm_cd,grp_nm,parm_desc,CASRN,SRSName,parm_unit
1646,00010,Physical,"Temperature, water, degrees Celsius",NaN,"Temperature, water",deg C
1660,00045,Physical,"Precipitation, total, inches",NaN,Precipitation,in
1673,00060,Physical,"Discharge, cubic feet per second",NaN,"Stream flow, mean. daily",ft3/s
1677,00065,Physical,"Gage height, feet",NaN,"Height, gage",ft
1692,00095,Physical,"Specific conductance, water, unfiltered, micro...",NaN,Specific conductance,uS/cm @25C
1733,00400,Physical,"pH, water, unfiltered, field, standard units",NaN,pH,std units
1739,00480,Physical,"Salinity, water, unfiltered, parts per thousand",NaN,Salinity,ppth
1950,62610,Physical,"Groundwater level above NGVD 1929, feet",NaN,NaN,ft
1951,62611,Physical,"Groundwater level above NAVD 1988, feet",NaN,NaN,ft
1999,63158,Physical,"Stream water level elevation above NGVD 1929, ...",NaN,NaN,ft


In [173]:
avail_parms_count_df = pd.merge(parm_instance_all, avail_parms_all, left_index=True, right_on='parm_cd')
avail_parms_count_df = avail_parms_count_df.rename(columns={'parm_cd_x': 'count_nu'})
avail_parms_count_df = avail_parms_count_df.drop(['parm_cd_y'], axis=1)
display(avail_parms_count_df)

,parm_cd,count_nu,grp_nm,parm_desc,CASRN,SRSName,parm_unit
1950,62610,245,Physical,"Groundwater level above NGVD 1929, feet",NaN,NaN,ft
1951,62611,245,Physical,"Groundwater level above NAVD 1988, feet",NaN,NaN,ft
2059,72019,245,Physical,"Depth to water level, feet below land surface",NaN,Depth to water level below land surface,ft
1646,00010,118,Physical,"Temperature, water, degrees Celsius",NaN,"Temperature, water",deg C
1692,00095,89,Physical,"Specific conductance, water, unfiltered, micro...",NaN,Specific conductance,uS/cm @25C
1677,00065,87,Physical,"Gage height, feet",NaN,"Height, gage",ft
1673,00060,67,Physical,"Discharge, cubic feet per second",NaN,"Stream flow, mean. daily",ft3/s
3421,99133,33,Nutrient,"Nitrate plus nitrite, water, in situ, milligra...",NaN,Inorganic nitrogen (nitrate and nitrite),mg/l as N
2740,00300,30,"Inorganics, Major, Non-metals","Dissolved oxygen, water, unfiltered, milligram...",7782-44-7,Oxygen,mg/l
1733,00400,30,Physical,"pH, water, unfiltered, field, standard units",NaN,pH,std units
